In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Загрузим библиотеки

In [ ]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV
from scipy.stats import norm
from scipy import stats

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

In [ ]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 14})

### Подгружаем данные

In [ ]:
TRAIN_DATASET_PATH = '../input/inputhttpswwwkagglecomevgeniyyudakov/train.csv'
TEST_DATASET_PATH = '../input/inputhttpswwwkagglecomevgeniyyudakov/test.csv'

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)

train_df

In [ ]:
test_df = pd.read_csv(TEST_DATASET_PATH)

test_df

### Уменьшаем размер данных для лучшей работы модели

In [ ]:
def reduce_mem_usage(train_df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = train_df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in train_df.columns:
        col_type = train_df[col].dtype
        
        if col_type != object:
            c_min = train_df[col].min()
            c_max = train_df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_df[col] = train_df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_df[col] = train_df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_df[col] = train_df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_df[col] = train_df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_df[col] = train_df[col].astype(np.float32)
                else:
                    train_df[col] = train_df[col].astype(np.float64)
        else:
            train_df[col] = train_df[col].astype('category')

    end_mem = train_df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return train_df

In [ ]:
def reduce_mem_usage(test_df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = test_df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in test_df.columns:
        col_type = test_df[col].dtype
        
        if col_type != object:
            c_min = test_df[col].min()
            c_max = test_df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    test_df[col] = test_df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    test_df[col] = test_df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    test_df[col] = test_df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    test_df[col] = test_df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    test_df[col] = test_df[col].astype(np.float32)
                else:
                    test_df[col] = test_df[col].astype(np.float64)
        else:
            test_df[col] = test_df[col].astype('category')

    end_mem = test_df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return test_df

### Проверим тип данных

In [ ]:
train_df.dtypes

### Мы видим признаки id и District id, мы их учитывать не будем, поэтому меняем тип данных на str

In [ ]:
train_df['Id'] = train_df['Id'].astype(str)
train_df['DistrictId'] = train_df['DistrictId'].astype(str)

In [ ]:
def df_fix_room(train_df):
    info_by_district_id = train_df.groupby(['DistrictId', 'HouseYear'], as_index=False).agg(
        {'Rooms': 'sum', 'Square': 'sum'}).rename(
        columns={'Rooms': 'sum_roos_dr', 'Square': 'sum_square_dr'})

    info_by_district_id['mean_square_per_room_in_dr'] = info_by_district_id['sum_square_dr'] \
        / info_by_district_id['sum_roos_dr']
    info_by_district_id.drop(
        ['sum_square_dr', 'sum_roos_dr'], axis=1, inplace=True)

    train_df = pd.merge(train_df, info_by_district_id, on=[
                  'DistrictId', 'HouseYear'], how='left')

    train_df['mean_square_per_room_in_dr'] = train_df['mean_square_per_room_in_dr'].fillna(
        train_df['mean_square_per_room_in_dr'].mean())

    train_df.loc[train_df['Rooms'] > 6, 'Rooms'] \
        = (train_df.loc[train_df['Rooms'] > 6, 'Square']
           // train_df.loc[train_df['Rooms'] > 6, 'mean_square_per_room_in_dr']).astype('int')

    train_df.loc[df['Rooms'] == 0, 'Rooms'] \
        = (train_df.loc[train_df['Rooms'] == 0, 'Square']
           // train_df.loc[train_df['Rooms'] == 0, 'mean_square_per_room_in_dr']).astype('int')

    train_df.loc[train_df['Rooms'] == 0, 'Rooms'] = 1
    return train_df

### Проверим выполнение

In [ ]:
train_df.describe()

### Проведём анализ данных и приведём их к нормальному виду

### Начнём с Rooms

In [ ]:
train_df['Rooms'].value_counts()

#### В Rooms мы видим значения равные 0. Будем считать эти значение как признак обозначающий квартиру "Студия". Для правильной работы модели заменим 0 на 1
#### Также мы видим, что в таблице имеются позиции со сверх количеством комнат(Например 19). Для правильной работы модели отредактируем это значение

In [ ]:
train_df['Rooms_outlier'] = 0
train_df.loc[(train_df['Rooms'] == 0) | (train_df['Rooms'] >= 6), 'Rooms_outlier'] = 1
train_df.head(20)

In [ ]:
train_df.loc[train_df['Rooms'] == 0, 'Rooms'] = 1
train_df.loc[train_df['Rooms'] >= 6, 'Rooms'] = train_df['Rooms'].median()

In [ ]:
train_df['Rooms'].value_counts()

#### Теперь проверим Square. Для анализа будем использовать boxplot. Это способ покажет нам данные выходящие за рамки.

In [ ]:
plt.figure(figsize = (16,3))
sns.boxplot(train_df['Square'], whis = 1.5)
plt.xlabel('Square')
plt.show()

### На графике мы видим выбросы, если эти данные оставить как есть, то мы получим некорректный max and min. 

In [ ]:
train_df.describe() ### Серийный 100 квартирный дом предусматривает 6 комнатные квартиры до 270 м2, а минимальная площадь примерно равна 16 м2

In [ ]:
train_df.loc[train_df['Square'] > 270].nlargest(30, 'Square')

In [ ]:
plt.figure(figsize = (16,3))
sns.boxplot(train_df['Square'], whis = 1.5)
plt.xlabel('Square')
plt.show()

In [ ]:
train_df.loc[train_df['Square'] >= 271, 'Square'] = train_df.loc[train_df['Square'] > 271, 'Square'] / 3 

In [ ]:
train_df.loc[train_df['Square'] <= 16, 'Square'] = (train_df.loc[train_df['Square'] <= 16, 'Square'] * 10) + 6 

# Где 6 - это минимальная площадь кухни

In [ ]:
plt.figure(figsize = (16,3))
sns.boxplot(train_df['Square'], whis = 1.5)
plt.xlabel('Square')
plt.show()

In [ ]:
train_df.describe() ### Серийный 100 квартирный дом предусматривает 6 комнатные квартиры до 270 м2, а минимальная площадь примерно равна 16 м2

### Посчитаем и заполним сразу KitchenSquare, так как мы ранее зафиксировали, что минимальная площадь кухни будет 6м2 и по стандартам сама KitchenSquare должна занимать 20% - 30% от всей Square. Остальное отдадим LifeSquare.

In [ ]:
plt.figure(figsize = (16, 8))

train_df['LifeSquare'].hist(bins=30)
plt.ylabel('Count')
plt.xlabel('LifeSquare')

plt.title('LifeSquare')
plt.show()

In [ ]:
# minLifeSquare
Optimum = train_df['LifeSquare'].median()
train_df['LifeSquare'].fillna(Optimum, inplace = True)

In [ ]:
train_df['LifeSquare'].isna().sum()

In [ ]:
train_df.loc[train_df['LifeSquare'] < 11, 'LifeSquare'] = 11

In [ ]:
# maxLifeSquare ограничим 150, так как это значение составляет 30% от maxSquare

train_df.loc[train_df['LifeSquare'] >= 150, 'LifeSquare'] = train_df.loc[train_df['LifeSquare'] >= 150, 'LifeSquare'] / 70 



In [ ]:
plt.figure(figsize = (16,3))
sns.boxplot(train_df['LifeSquare'], whis = 1.5)
plt.xlabel('LifeSquare')
plt.show()

### Проработав выбросы и заменив Nan  в LifeSquare получаем beatifull boxplot)

In [ ]:
train_df['KitchenSquare'].quantile(.975), train_df['KitchenSquare'].quantile(.025)

In [ ]:
condition = (train_df['KitchenSquare'].isna()) \
             | (train_df['KitchenSquare'] > train_df['KitchenSquare'].quantile(.975))
        
train_df.loc[condition, 'KitchenSquare'] = train_df['KitchenSquare'].median()

train_df.loc[train_df['KitchenSquare'] < 5, 'KitchenSquare'] = 5

In [ ]:
train_df.describe()

### Теперь проверим пропуски

In [ ]:
train_df.isna().sum()

### Выгрузим данные для проверки работы модели

In [ ]:
train_df[['Rooms','Square', 'LifeSquare', 'KitchenSquare']].head(20)

### Чтобы не проверять все данные обезапасим себя

In [ ]:
def prepare_lifesquare(train_df):
    train_df.loc[train_df['Square'] < train_df['LifeSquare'],
           'LifeSquare'] = train_df.loc[df['Square'] < train_df['LifeSquare'], 'Square']
    return train_df


def fillna_life_square(train_df):
    train_df['LifeSquare'] = train_df['LifeSquare'].fillna(train_df['LifeSquare'].mean())
    return df

### Т.е. если в нашей таблице в столбце Square будет больше чем в столбце LifeSquare, то произойдёт замена данных.

### HouseYear слишком большой и имеет некорректные данные. Приведём их к нормальному виду

In [ ]:
train_df.loc[train_df['HouseYear'] > 2020]

In [ ]:
train_df.loc[train_df['HouseYear'] == 20052011, 'HouseYear'] = int((2005 + 2011) / 2)
train_df.loc[train_df['HouseYear'] == 4968, 'HouseYear'] = 1968

### Просмотрев данные видим зависимость признаков друг от друга. Попробуем её найти

In [ ]:
train_df.loc[train_df['Healthcare_1'] == 1046]

In [ ]:
plt.figure(figsize = (16,3))
sns.boxplot(train_df['Healthcare_1'], whis = 1.5)
plt.xlabel('Healthcare_1')
plt.show()

### По признаку Healthcare_1 мы видим, что все значения идут в порядке от 0 до 5

In [ ]:
plt.figure(figsize = (16,3))
sns.boxplot(train_df['Helthcare_2'], whis = 1.5)
plt.xlabel('Helthcare_2')
plt.show()

По признаку Healthcare_2 мы также видим, что все значения идут в порядке от 0 до 5

Тогда получается, что есть некие подуровни/зоны здравохранения от 0 до 5. Преобразуем Healthcare_1 и Healthcare_2

In [ ]:
Healthcare = train_df['Healthcare_1'].median()
train_df['Healthcare_1'].fillna(Healthcare, inplace = True)
train_df['Healthcare_1'] = round((train_df['Healthcare_1'] // 100),0)

In [ ]:
train_df['Healthcare_1'].value_counts()

In [ ]:
train_df.loc[train_df['Healthcare_1'] >= 6, 'Healthcare_1'] = round((train_df.loc[train_df['Healthcare_1'] >= 6, 'Healthcare_1'] // 10),0) 
train_df['Healthcare_1'] = train_df['Healthcare_1'].astype(np.int64)

In [ ]:
train_df.head(20)

In [ ]:
plt.figure(figsize = (16,3))
sns.boxplot(train_df['Healthcare_1'], whis = 1.5)
plt.xlabel('Healthcare_1')
plt.show()

In [ ]:
train_df.head(20)

In [ ]:
train_df.loc[train_df['Helthcare_2'] >= 5, 'Helthcare_2'] = 5

In [ ]:
train_df['Helthcare_2'] = train_df['Helthcare_2'].astype(np.int64)

In [ ]:
train_df.describe()

### Ecology_1 Доля площади земель лесного фонда от площади Social

In [ ]:
train_df['Ecology_1'] = round(train_df['Ecology_1']*100)

In [ ]:
train_df.describe()

In [ ]:
train_df[['Ecology_1', 'Healthcare_1', 'Social_1','Rooms','Square', 'Price']]

In [ ]:
train_df['HouseFloor'].sort_values().unique()

In [ ]:
train_df['Floor'].sort_values().unique()

In [ ]:
(train_df['Floor'] > train_df['HouseFloor']).sum()

In [ ]:
train_df['HouseFloor_outlier'] = 0
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
train_df.loc[train_df['Floor'] > train_df['HouseFloor'], 'HouseFloor_outlier'] = 1

In [ ]:
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor'] = train_df['HouseFloor'].median()

In [ ]:
floor_outliers = train_df.loc[train_df['Floor'] > train_df['HouseFloor']].index
floor_outliers

In [ ]:
train_df.loc[floor_outliers, 'Floor'] = train_df.loc[floor_outliers, 'HouseFloor']\
                                                .apply(lambda x: random.randint(1, x))

In [ ]:
(train_df['Floor'] > train_df['HouseFloor']).sum()

In [ ]:
class DataPreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Параметры класса"""
        self.medians=None
        self.kitchen_square_quantile = None
        
        
    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median()
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
        
        
    
    def transform(self, X):
        """Трансформация данных"""

        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 6, 'Rooms'] = self.medians['Rooms']
        
        
        # KitchenSquare
        condition = (X['KitchenSquare'].isna()) \
                    | (X['KitchenSquare'] > self.kitchen_square_quantile)
        
        X.loc[condition, 'KitchenSquare'] = self.medians['KitchenSquare']

        X.loc[X['KitchenSquare'] < 5, 'KitchenSquare'] = 5
        
        # HouseFloor, Floor
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        
        floor_outliers = X.loc[X['Floor'] > X['HouseFloor']].index
        X.loc[floor_outliers, 'Floor'] = X.loc[floor_outliers, 'HouseFloor']\
                                            .apply(lambda x: random.randint(1, x))
        
        # HouseYear
        current_year = datetime.now().year
        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year
        
        # Healthcare_1
        Healthcare = self.medians['Healthcare_1']
        X['Healthcare_1'].fillna(Healthcare, inplace = True)
        X['Healthcare_1'] = round((X['Healthcare_1'] // 100),0)
        X.loc[train_df['Healthcare_1'] >= 6, 'Healthcare_1'] = round((X.loc[train_df['Healthcare_1'] >= 6, 'Healthcare_1'] // 10),0) 
        X['Healthcare_1'] = X['Healthcare_1'].astype(np.int64)
            
        # Ecology_1
        X['Ecology_1'] = round(X['Ecology_1']*100)
        
        # Square
        X.loc[train_df['Square'] >= 271, 'Square'] = X.loc[X['Square'] > 271, 'Square'] / 3
        X.loc[train_df['Square'] <= 16, 'Square'] = (X.loc[X['Square'] <= 16, 'Square'] * 10) + 6 
            
        # LifeSquare
        Optimum = self.medians['LifeSquare']
        X['LifeSquare'].fillna(Optimum, inplace = True)
        X.loc[X['LifeSquare'] < 11, 'LifeSquare'] = 11
        X.loc[X['LifeSquare'] >= 150, 'LifeSquare'] = X.loc[X['LifeSquare'] >= 150, 'LifeSquare'] / 70
        
        
        X.fillna(self.medians, inplace=True)
        
        return X

4. Построение новых признаков 

In [ ]:
binary_to_numbers = {'A': 0, 'B': 1}

train_df['Ecology_2'] = train_df['Ecology_2'].replace(binary_to_numbers)
train_df['Ecology_3'] = train_df['Ecology_3'].replace(binary_to_numbers)
train_df['Shops_2'] = train_df['Shops_2'].replace(binary_to_numbers)

In [ ]:
district_size = train_df['DistrictId'].value_counts().reset_index()\
                    .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size.head()

In [ ]:
train_df = train_df.merge(district_size, on='DistrictId', how='left')
train_df.head()

In [ ]:
train_df['IsDistrictLarge'] = (train_df['DistrictSize'] > 100).astype(int)

In [ ]:
med_price_by_district = train_df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                            .rename(columns={'Price':'MedPriceByDistrict'})

med_price_by_district.head()

In [ ]:
train_df = train_df.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
train_df.head()

In [ ]:
def floor_to_cat(X):

    X['floor_cat'] = 0

    X.loc[X['Floor'] <= 3, 'floor_cat'] = 1  
    X.loc[(X['Floor'] > 3) & (X['Floor'] <= 5), 'floor_cat'] = 2
    X.loc[(X['Floor'] > 5) & (X['Floor'] <= 9), 'floor_cat'] = 3
    X.loc[(X['Floor'] > 9) & (X['Floor'] <= 15), 'floor_cat'] = 4
    X.loc[X['Floor'] > 15, 'floor_cat'] = 5

    return X


def floor_to_cat_pandas(X):
    bins = [0, 3, 5, 9, 15, X['Floor'].max()]
    X['floor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)
    
    X['floor_cat'].fillna(-1, inplace=True)
    return X


def year_to_cat(X):

    X['year_cat'] = 0

    X.loc[X['HouseYear'] <= 1941, 'year_cat'] = 1
    X.loc[(X['HouseYear'] > 1941) & (X['HouseYear'] <= 1945), 'year_cat'] = 2
    X.loc[(X['HouseYear'] > 1945) & (X['HouseYear'] <= 1980), 'year_cat'] = 3
    X.loc[(X['HouseYear'] > 1980) & (X['HouseYear'] <= 2000), 'year_cat'] = 4
    X.loc[(X['HouseYear'] > 2000) & (X['HouseYear'] <= 2010), 'year_cat'] = 5
    X.loc[(X['HouseYear'] > 2010), 'year_cat'] = 6

    return X


def year_to_cat_pandas(X):
    bins = [0, 1941, 1945, 1980, 2000, 2010, X['HouseYear'].max()]
    X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)
    
    X['year_cat'].fillna(-1, inplace=True)
    return X

In [ ]:
bins = [0, 3, 5, 9, 15, train_df['Floor'].max()]
pd.cut(train_df['Floor'], bins=bins, labels=False)

In [ ]:
bins = [0, 3, 5, 9, 15, train_df['Floor'].max()]
pd.cut(train_df['Floor'], bins=bins)

In [ ]:
train_df = year_to_cat(train_df)
train_df = floor_to_cat(train_df)
train_df.head()

In [ ]:
med_price_by_floor_year = train_df.groupby(['year_cat', 'floor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByFloorYear'})
med_price_by_floor_year.head()

In [ ]:
train_df = train_df.merge(med_price_by_floor_year, on=['year_cat', 'floor_cat'], how='left')
train_df.head()

In [ ]:
class FeatureGenetator():
    """Генерация новых фич"""
    
    def __init__(self):
        self.DistrictId_counts = None
        self.binary_to_numbers = None
        self.med_price_by_district = None
        self.med_price_by_floor_year = None
        self.house_year_max = None
        self.floor_max = None
        self.district_size = None


        
    def fit(self, X, y=None):
        
        X = X.copy()
        
        # Binary features
        self.binary_to_numbers = {'A': 0, 'B': 1}
        
        # DistrictID
        self.district_size = X['DistrictId'].value_counts().reset_index() \
                               .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})
                
        # Target encoding
        ## District, Rooms
        df = X.copy()
        
        if y is not None:
            df['Price'] = y.values
            
            self.med_price_by_district = df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                                            .rename(columns={'Price':'MedPriceByDistrict'})
            
            self.med_price_by_district_median = self.med_price_by_district['MedPriceByDistrict'].median()
            
        ## floor, year
        if y is not None:
            self.floor_max = df['Floor'].max()
            self.house_year_max = df['HouseYear'].max()
            df['Price'] = y.values
            df = self.floor_to_cat(df)
            df = self.year_to_cat(df)
            self.med_price_by_floor_year = df.groupby(['year_cat', 'floor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByFloorYear'})
            self.med_price_by_floor_year_median = self.med_price_by_floor_year['MedPriceByFloorYear'].median()
        

        
    def transform(self, X):
        
        
        # Binary features
        X['Ecology_2'] = X['Ecology_2'].map(self.binary_to_numbers)  # self.binary_to_numbers = {'A': 0, 'B': 1}
        X['Ecology_3'] = X['Ecology_3'].map(self.binary_to_numbers)
        X['Shops_2'] = X['Shops_2'].map(self.binary_to_numbers)
        
        # DistrictId, IsDistrictLarge
        X = X.merge(self.district_size, on='DistrictId', how='left')
        
        X['new_district'] = 0
        X.loc[X['DistrictSize'].isna(), 'new_district'] = 1
        
        X['DistrictSize'].fillna(5, inplace=True)
        
        X['IsDistrictLarge'] = (X['DistrictSize'] > 100).astype(int)
        
        # More categorical features
        X = self.floor_to_cat(X)  # + столбец floor_cat
        X = self.year_to_cat(X)   # + столбец year_cat
        
        # Target encoding
        if self.med_price_by_district is not None:
            X = X.merge(self.med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
            X.fillna(self.med_price_by_district_median, inplace=True)
            
        if self.med_price_by_floor_year is not None:
            X = X.merge(self.med_price_by_floor_year, on=['year_cat', 'floor_cat'], how='left')
            X.fillna(self.med_price_by_floor_year_median, inplace=True)
        
        return X
    
    def floor_to_cat(self, X):
        bins = [0, 3, 5, 9, 15, self.floor_max]
        X['floor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)

        X['floor_cat'].fillna(-1, inplace=True) 
        return X
     
    def year_to_cat(self, X):
        bins = [0, 1941, 1945, 1980, 2000, 2010, self.house_year_max]
        X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)

        X['year_cat'].fillna(-1, inplace=True)
        return X
            

In [ ]:
train_df.columns.tolist()

In [ ]:
feature_names = ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                 'Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 
                 'Helthcare_2', 'Shops_1', 'Shops_2'] 

new_feature_names = ['Rooms_outlier'] 

target_name = 'Price'

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

X = train_df.drop(columns=target_name)
y = train_df[target_name]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=21)

In [ ]:
preprocessor = DataPreprocessing()
preprocessor.fit(X_train)

X_train = preprocessor.transform(X_train)
X_valid = preprocessor.transform(X_valid)
test_df = preprocessor.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [ ]:
features_gen = FeatureGenetator()
features_gen.fit(X_train, y_train)

X_train = features_gen.transform(X_train)
X_valid = features_gen.transform(X_valid)
test_df = features_gen.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [ ]:
X_train = X_train[feature_names + new_feature_names]
X_valid = X_valid[feature_names + new_feature_names]
test_df = test_df[feature_names + new_feature_names]

In [ ]:
X_train.isna().sum().sum(), X_valid.isna().sum().sum(), test_df.isna().sum().sum()

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.ensemble import BaggingRegressor

gb = GradientBoostingRegressor(
                               max_depth=4,
                               min_samples_leaf=20,
                               random_state=21,  
                               n_estimators=150
                              )

In [ ]:
%%time
rf = RandomForestRegressor(
    random_state=21, 
    max_depth=40,  # gridsearch
    criterion='mse',
    min_samples_leaf=5,  # gridsearch
    n_jobs=-1,
    n_estimators=1000  # gridsearch
)
# rf.fit(X_train, y_train)

In [ ]:
%%time
rf2 = RandomForestRegressor(
    random_state=21, 
    max_depth=17, 
    criterion='mse',
    max_features=7, 
    n_jobs=-1,
    n_estimators=200  
)
# rf.fit(X_train, y_train)

In [ ]:
%%time
rf3 = RandomForestRegressor(
    max_depth=20,
    random_state=21, 
)

In [ ]:
%%time
rf4 = RandomForestRegressor(
    random_state=21, 
)

In [ ]:
%%time
import xgboost as xgb

xg=xgb.XGBRegressor(
                  random_state=21,
                  n_estimators=5, 
                  n_jobs=-1,
                  subsample=0.5,
                  colsample_bynode=0.5,
                  num_parallel_tree=100,
                  learning_rate=0.5,
                  max_depth=3
                  )

In [ ]:
%%time
from sklearn.ensemble import StackingRegressor

stack = StackingRegressor([
                           ('rf', rf),
                           ('rf2', rf2),
#                            ('rf3', rf3),
                           ('rf4', rf4),
                           ('gb', gb), 
                           ('xg', xg),
                          ],
                          cv=5,
                          n_jobs=-1,
                          final_estimator=GradientBoostingRegressor(
                               
                               max_depth=1,
                               random_state=21,  
                               n_estimators=85,
                               
                          )
                         )
                             
stack.fit(X_train, y_train)

In [ ]:
final_model = stack

In [ ]:
%%time
y_train_preds = final_model.predict(X_train)
y_test_preds = final_model.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

In [ ]:
test_df.shape, test_df

In [ ]:
test_df

In [ ]:
submit = pd.read_csv('../input/sample/sample_submission.csv')
submit.head()

In [ ]:
predictions = final_model.predict(test_df)
predictions

In [ ]:
submit['Price'] = predictions
submit.head()

In [ ]:
submit.to_csv('rf_submit.csv', index=False)